In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [ ]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_events')
except Exception as e:
    print(e)

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Create a session info table that will contain neccessary information
query = "CREATE TABLE IF NOT EXISTS session_info"
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_info (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
## Add in the SELECT statement to verify the data was entered into the table

query = "select artist, song_title, song_length from session_info WHERE session_id = 338 and item_in_session = 4"
try:
    session_rows = session.execute(query)
except Exception as e:
    print(e)
    
session_info_table = PrettyTable(['Artist', 'Song Title', 'Song Length'])
for row in session_rows:
    session_info_table.add_row([row.artist, row.song_title, row.song_length])
print(session_info_table)

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Create a user info table that will contain neccessary information
query = "CREATE TABLE IF NOT EXISTS user_info"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#Inserting the data to the user_info table    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_info (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Answering the second question
query = "select artist, song_title, first_name, last_name from user_info WHERE user_id = 10 and session_id = 182"
try:
    user_info_rows = session.execute(query)
except Exception as e:
    print(e)
    
user_info_table = PrettyTable(['Artist', 'Song Title', 'First Name', 'Last Name'])
for row in user_info_rows:
    user_info_table.add_row([row.artist, row.song_title, row.first_name, row.last_name])
print(user_info_table)

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create a user info table that will contain neccessary information
query = "CREATE TABLE IF NOT EXISTS user_data"
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY ((song_title), user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#Inserting the data to the user_info table    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_data (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# Answering the second question
query = "select first_name, last_name from user_data WHERE song_title='All Hands Against His Own'"
try:
    user_data_rows = session.execute(query)
except Exception as e:
    print(e)
    
user_data_table = PrettyTable(['First Name', 'Last Name'])
for row in user_data_rows:
    user_data_table.add_row([row.first_name, row.last_name])
print(user_data_table)

In [ ]:
## Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS session_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_data"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
session.shutdown()
cluster.shutdown()